In [1]:
import pandas as pd
import matplotlib.pyplot as plt



In [2]:
from src.model import *

In [3]:
SPECTRA_DIR = 'Bruker_MPA/*'

AFSIS_PATH = Path('afsis/2009-2013/Dry_Chemistry/ICRAF/')

spectra_df = load_spectra_df(SPECTRA_DIR, AFSIS_PATH)


In [4]:
wet_chem_df = pd.read_csv('afsis/2009-2013/Wet_Chemistry/CROPNUTS/Wet_Chemistry_CROPNUTS.csv')
wet_chem_df.head()

,SSN,Public,EC,ExAc,ExBas,M3 Al,M3 B,M3 Ca,M3 Cu,M3 Fe,M3 K,M3 Mg,M3 Mn,M3 Na,M3 P,M3 S,M3 Zn,PH,PSI
0,icr006475,True,0.053,2.56,2.744241,1095.0,0.3337,207.1,1.3150,153.70,306.30,89.5,9.266,42.71,4.495,18.960,2.825,4.682,67.052495
1,icr006586,True,0.193,0.59,16.797753,1165.0,0.7899,1665.0,2.8140,126.80,1186.00,636.7,254.700,40.83,12.510,13.600,1.952,7.062,30.420552
2,icr007929,True,0.305,0.19,16.503619,727.6,0.6142,2518.0,0.2496,260.40,72.57,428.5,35.820,42.94,21.090,14.810,2.742,7.114,60.506163
3,icr008008,True,0.053,0.48,6.302775,1458.0,0.3603,734.3,1.3690,134.10,274.60,222.5,29.110,20.73,109.200,11.400,0.980,5.650,51.470143
4,icr010198,True,0.032,1.05,2.390303,2166.0,0.0100,261.8,1.6860,97.49,91.76,76.5,375.200,49.31,3.958,5.281,1.258,5.501,247.626533


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
WET_CHEM_PATH = 'afsis/2009-2013/Wet_Chemistry/CROPNUTS/Wet_Chemistry_CROPNUTS.csv'
elements = ['M3 Ca', 'M3 K', 'M3 Mg', 'M3 P']
columns_to_load = elements + ['SSN']


wet_chem_df = pd.read_csv(WET_CHEM_PATH, index_col='SSN', usecols=columns_to_load)

In [7]:
data_df = spectra_df.merge(wet_chem_df, left_index=True, right_index=True)

In [8]:
y = data_df[['M3 Ca', 'M3 K', 'M3 Mg', 'M3 P']]

In [23]:
y_mask = y.notnull()
y = y[y_mask]
y = np.log1p(y)

X = spectra_df[y_mask]
X = np.apply_along_axis(np.gradient, 1, X)

X_train, X_validate, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
Y_COLUMN = 'M3 K'

m3_k_xgb = basic_xgboost(spectra_df, WET_CHEM_PATH, Y_COLUMN)

Training model for M3 K.
1907 samples available.


KeyboardInterrupt: 

In [10]:
Y_COLUMN = 'M3 Ca'

m3_ca_xgb = basic_xgboost2(X_train, y_train[Y_COLUMN])

Training model for M3 Ca.
1277 samples available.
CV r^2 score: 0.7067729410303515


In [24]:


m3_ca_xgb.fit(X_train, y_train[Y_COLUMN])

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=20, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=-1, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [25]:
yhat = m3_ca_xgb.predict(X_validate)

In [18]:
from sklearn.metrics import mean_squared_error

In [26]:
np.sqrt(mean_squared_error(y_test[Y_COLUMN], yhat))

0.1909289424007054

In [ ]:
Y_COLUMN = 'M3 Ca'

m3_ca_xgb = basic_xgboost3(X_train, y_train[Y_COLUMN])